In [1]:
import os, sys
import dotenv

import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm

sys.path.append(os.environ.get('PATH_OPENSKY'))
from user_eval import get_df_posts 
from news_outlet_eval import get_news_feed, get_posts_from_nos

dotenv.load_dotenv('.env')    
PATH_DATA = os.environ.get('PATH_OPENSKY') + os.environ.get('PATH_REL_DATA')
PATH_USER_POSTS = PATH_DATA + 'user_posts/'

PATH_RESULTS = os.environ.get('PATH_OPENSKY') + 'results/'
PATH_USER_REPLIES_NEWS = PATH_RESULTS + 'agg_user_replies_news_2.csv'
PATH_USER_REPLIES_NEWS_POSTS = PATH_RESULTS + 'agg_user_replies_news_posts_2.csv'

PATH_FEED_POSTS = PATH_DATA + 'feed_posts/'

df = pd.read_csv(PATH_USER_REPLIES_NEWS_POSTS)

/tmp/ipykernel_281663/345954859.py:23: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH_USER_REPLIES_NEWS_POSTS)


In [2]:
df

,post_id,user_id,instance,date,text,langs,like_count,reply_count,repost_count,reply_to,replied_author,thread_root,thread_root_author,repost_from,reposted_author,quotes,quoted_author,labels,sent_label,sent_score
0,10657080,284413,bsky.social,2024-01-14 08:03:00,let’s hope his expands ten fold 💪💪,['eng'],0,0,0,10657081.0,15717.0,10657081.0,15717.0,NaN,NaN,NaN,NaN,NaN,2.0,0.945
1,10657084,284413,bsky.social,2024-01-12 22:12:00,"yes, we can do this! 💪",['eng'],3,0,0,5595208.0,2486.0,5595208.0,2486.0,NaN,NaN,NaN,NaN,NaN,2.0,0.956
2,10657085,284413,bsky.social,2024-01-11 14:19:00,"in other words, “as long as it takes” it refer...",['eng'],0,1,0,10657086.0,12030.0,10657086.0,12030.0,NaN,NaN,NaN,NaN,NaN,1.0,0.583
3,10657096,284413,bsky.social,2024-01-08 14:10:00,i’ll believe it when i see the weapons in Ukra...,['eng'],0,0,0,10657097.0,15717.0,10657097.0,15717.0,NaN,NaN,NaN,NaN,NaN,1.0,0.509
4,10657117,284413,bsky.social,2024-01-05 09:22:00,Expect the world to become way way worse. If a...,['eng'],1,0,0,5271912.0,15717.0,5271912.0,15717.0,NaN,NaN,NaN,NaN,NaN,0.0,0.890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36816,171050451,141248,bsky.social,2024-03-09 13:47:00,La preuve qu'ils sont antidémocratiques. \nPer...,"['eng', 'fra']",4,0,0,5550653.0,4890.0,5550653.0,4890.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36817,105014318,92560,bsky.social,2024-03-08 01:59:00,What a complete abdication of good journalism ...,['eng'],0,0,0,698597.0,9.0,698597.0,9.0,NaN,NaN,NaN,NaN,NaN,0.0,0.790
36818,105014322,92560,bsky.social,2024-03-07 22:16:00,Trump and Biden are at par with age...exactly ...,['eng'],4,0,0,127576.0,379.0,127576.0,379.0,NaN,NaN,NaN,NaN,NaN,0.0,0.867
36819,105014501,92560,bsky.social,2024-01-05 08:08:00,Because doctors are literally refusing to pres...,['eng'],1,0,0,145919.0,379.0,145919.0,379.0,NaN,NaN,NaN,NaN,NaN,0.0,0.882


In [3]:
relevant_news_items = df.thread_root.unique()
df_news_feed = get_news_feed()
df_news_feed = df_news_feed[df_news_feed.post_id.isin(relevant_news_items)]
df_news_feed['news_item'] = True
df['news_item'] = False
df_news_feed

,post_id,user_id,instance,date,text,langs,like_count,reply_count,repost_count,reply_to,replied_author,thread_root,thread_root_author,quotes,quoted_author,labels,news_item
0,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,NaN,NaN,NaN,None,None,None,True
1,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,NaN,NaN,NaN,None,None,None,True
2,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,NaN,NaN,NaN,None,None,None,True
4,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,NaN,NaN,NaN,None,None,None,True
5,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,NaN,NaN,NaN,None,None,None,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42100,8540816,319,com,2023-06-30 19:39:00,Decades before he was among the Supreme Court ...,[eng],87,6,21,NaN,NaN,NaN,NaN,None,None,None,True
42103,1233312,319,com,2023-06-28 17:12:00,Prosecutors accuse Trump of showing off classi...,None,26,6,8,NaN,NaN,NaN,NaN,None,None,None,True
42106,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,NaN,NaN,NaN,None,None,None,True
42109,44325423,319,com,2023-06-21 18:43:00,Florida Gov. Ron DeSantis’s wealthy donors and...,None,19,6,6,NaN,NaN,NaN,NaN,None,None,None,True


In [4]:
df_news_feed['news_item'] = True
df['news_item'] = False 
df = pd.concat([df_news_feed, df])
df

/tmp/ipykernel_281663/1405401380.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_news_feed, df])


,post_id,user_id,instance,date,text,langs,like_count,reply_count,repost_count,reply_to,...,thread_root,thread_root_author,quotes,quoted_author,labels,news_item,repost_from,reposted_author,sent_label,sent_score
0,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,...,NaN,NaN,NaN,NaN,None,True,NaN,NaN,NaN,NaN
1,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,...,NaN,NaN,NaN,NaN,None,True,NaN,NaN,NaN,NaN
2,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,...,NaN,NaN,NaN,NaN,None,True,NaN,NaN,NaN,NaN
4,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,...,NaN,NaN,NaN,NaN,None,True,NaN,NaN,NaN,NaN
5,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,...,NaN,NaN,NaN,NaN,None,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36816,171050451,141248,bsky.social,2024-03-09 13:47:00,La preuve qu'ils sont antidémocratiques. \nPer...,"['eng', 'fra']",4,0,0,5550653.0,...,5550653.0,4890.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
36817,105014318,92560,bsky.social,2024-03-08 01:59:00,What a complete abdication of good journalism ...,['eng'],0,0,0,698597.0,...,698597.0,9.0,NaN,NaN,NaN,False,NaN,NaN,0.0,0.790
36818,105014322,92560,bsky.social,2024-03-07 22:16:00,Trump and Biden are at par with age...exactly ...,['eng'],4,0,0,127576.0,...,127576.0,379.0,NaN,NaN,NaN,False,NaN,NaN,0.0,0.867
36819,105014501,92560,bsky.social,2024-01-05 08:08:00,Because doctors are literally refusing to pres...,['eng'],1,0,0,145919.0,...,145919.0,379.0,NaN,NaN,NaN,False,NaN,NaN,0.0,0.882


In [5]:
df.drop_duplicates(subset='post_id', inplace=True)
df

,post_id,user_id,instance,date,text,langs,like_count,reply_count,repost_count,reply_to,...,thread_root,thread_root_author,quotes,quoted_author,labels,news_item,repost_from,reposted_author,sent_label,sent_score
0,3517724,27155,com,2023-10-20 11:25:00,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,...,NaN,NaN,NaN,NaN,None,True,NaN,NaN,NaN,NaN
7,3753243,379,com,2024-03-18 23:41:00,"A conservative social media influencer, Isabel...",[eng],153,12,26,NaN,...,NaN,NaN,NaN,NaN,None,True,NaN,NaN,NaN,NaN
16,697660,379,com,2024-03-18 23:06:00,Donald Trump accused Jewish voters of hating t...,[eng],55,16,18,NaN,...,NaN,NaN,NaN,NaN,None,True,NaN,NaN,NaN,NaN
18,10705540,9,org,2024-03-18 23:03:00,More than 50 other countries have already bann...,[eng],150,9,32,NaN,...,NaN,NaN,NaN,NaN,None,True,NaN,NaN,NaN,NaN
27,936433,9,org,2024-03-18 22:21:00,The U.S. Supreme Court on Monday declined to s...,[eng],217,22,34,NaN,...,NaN,NaN,NaN,NaN,None,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36816,171050451,141248,bsky.social,2024-03-09 13:47:00,La preuve qu'ils sont antidémocratiques. \nPer...,"['eng', 'fra']",4,0,0,5550653.0,...,5550653.0,4890.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
36817,105014318,92560,bsky.social,2024-03-08 01:59:00,What a complete abdication of good journalism ...,['eng'],0,0,0,698597.0,...,698597.0,9.0,NaN,NaN,NaN,False,NaN,NaN,0.0,0.790
36818,105014322,92560,bsky.social,2024-03-07 22:16:00,Trump and Biden are at par with age...exactly ...,['eng'],4,0,0,127576.0,...,127576.0,379.0,NaN,NaN,NaN,False,NaN,NaN,0.0,0.867
36819,105014501,92560,bsky.social,2024-01-05 08:08:00,Because doctors are literally refusing to pres...,['eng'],1,0,0,145919.0,...,145919.0,379.0,NaN,NaN,NaN,False,NaN,NaN,0.0,0.882


In [6]:
df[['user_id', 'instance']].drop_duplicates().instance.value_counts().head(20)

instance
bsky.social    8719
com             184
de               35
net              29
org              28
dev              22
xyz              15
gay              15
fr               14
me               11
ca                8
social            7
us                7
eu                6
art               5
io                5
in                5
ch                4
blue              4
tv                4
Name: count, dtype: int64

In [7]:
df.to_csv(PATH_RESULTS + 'news_items_and_replies.csv', index=False)